In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import duckdb

base_dir = Path('c:\\Users\\henry\\OneDrive\\Personal Career\\Personal Projects\\GitHub\\Revenue-Sustainability-Analysis')
data_dir = Path(base_dir / 'Dataset')

## Define adopters vs. non adopters


In [2]:
os.chdir(data_dir)

df = pd.read_parquet('feature_usage.parquet')

In [3]:
df['usage_id'] = df['usage_id'].str.replace('syn_', "", regex=False)
df['usage_pk'] = df['usage_pk'].str.replace('sdv-',"", regex=False)

In [4]:
feat = df[df['feature_name'] == 'feature_12']

In [5]:
pd.to_datetime(feat['usage_date'])

12             NaT
17      2024-01-12
79      2024-07-15
82      2025-02-14
117     2024-04-11
           ...    
44803   2025-03-24
44851   2024-02-13
44947          NaT
44968   2024-05-08
44986   2025-03-06
Name: usage_date, Length: 1153, dtype: datetime64[ns]

In [6]:
pd.set_option('display.max_rows', 10)

### Adjust the release date of feature 12 to be June 12, more accurate of an actual release date

In [7]:
# delta = pd.Timestamp("2023-06-12") - pd.Timestamp("2023-01-01") 

# df['usage_date'] = pd.to_datetime(df['usage_date']) + delta

# df.to_parquet(data_dir / "feature_usage.parquet")

In [8]:
feat.groupby("subscription_id")['usage_date'] \
    .nunique() \
    .ge(1) \
    .astype(int) \
    .reset_index(name="adoption_flag")['adoption_flag'] \
    .value_counts()

adoption_flag
1    751
0    165
Name: count, dtype: int64

In [9]:
feat['subscription_id'].nunique()

916

In [17]:
subs = pd.read_parquet(data_dir / 'subscriptions.parquet')
accts = pd.read_parquet(data_dir / 'accounts.parquet')

In [19]:
subs.columns

Index(['subscription_id', 'account_id', 'start_date', 'end_date', 'plan_tier',
       'seats', 'mrr_amount', 'arr_amount', 'is_trial', 'upgrade_flag',
       'downgrade_flag', 'churn_flag', 'billing_frequency', 'auto_renew_flag'],
      dtype='object')

In [20]:
print(f"subs:{subs['subscription_id'].nunique()}")
print(f"accts:{subs['account_id'].nunique()}")

subs:4527
accts:500


In [21]:
print(f"accts:{accts['account_id'].nunique()}")

accts:458


In [23]:
accts['seats'].sum()

9287

In [ ]:
accts['account_id'].nunique()

458

In [30]:
seats_without_sub = subs[subs["subscription_id"].isna()]["seats"].sum()

seats_with_sub = subs["seats"].sum()

seats_without_sub / seats_with_sub

0.10304397012219334

In [32]:
subs

,subscription_id,account_id,start_date,end_date,plan_tier,seats,mrr_amount,arr_amount,is_trial,upgrade_flag,downgrade_flag,churn_flag,billing_frequency,auto_renew_flag
0,S-8cec59,A-3c1a3f,2023-12-23,2024-04-12,Enterprise,14,2786,33432,True,True,True,True,Monthly,True
1,S-0f6f44,A-9b9fe9,2024-06-11,None,Pro,17,833,9996,True,True,True,True,None,True
2,S-51c0d1,A-659280,2024-11-25,None,Enterprise,62,0,0,True,True,True,True,Annual,True
3,S-f81687,A-e7a1e2,2024-11-23,2024-12-13,Enterprise,5,995,11940,True,True,True,True,Monthly,True
4,S-cff5a2,A-ba6516,2024-01-10,None,Enterprise,27,5373,64476,True,True,True,True,Monthly,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,S-c158df,A-fa2041,2024-10-27,None,Enterprise,18,3582,42984,True,True,True,True,Monthly,True
4996,S-b49c0a,None,2024-12-05,None,Pro,23,1127,13524,True,True,True,True,Monthly,True
4997,S-42822d,A-443f6f,2024-12-15,None,Pro,25,1225,<NA>,True,True,True,True,Annual,True
4998,S-432401,A-c7ffc2,2023-05-25,None,Pro,<NA>,6468,77616,True,True,True,True,Monthly,True


In [31]:
feat

,usage_id,subscription_id,usage_date,feature_name,usage_count,usage_duration_secs,error_count,is_beta_feature,post_release,usage_pk
12,usage_12,S-56886a,NaT,feature_12,5,1395,0,True,False,id-dAMjQh
17,usage_17,None,2024-01-12,feature_12,12,338,1,True,False,id-cLtULh
79,usage_79,S-fb9972,2024-07-15,feature_12,9,1221,0,True,False,id-JWVxPF
82,usage_82,S-07399c,2025-02-14,feature_12,6,1050,1,True,False,id-ReBIia
117,usage_117,S-13a948,2024-04-11,feature_12,15,1112,0,True,False,id-QJxgAo
...,...,...,...,...,...,...,...,...,...,...
44803,usage_44803,S-e57a7d,2025-03-24,feature_12,10,1154,4,True,False,id-rPffQa
44851,usage_44851,S-f553b5,2024-02-13,feature_12,7,1202,0,True,False,id-ChXsRI
44947,usage_44947,S-0cb6ed,NaT,feature_12,11,180,0,True,False,id-nfuJHq
44968,usage_44968,S-c77f0c,2024-05-08,feature_12,17,1061,2,True,False,id-dKTZxJ
